In [2]:
import numpy as np
import xml.etree.ElementTree as ET
import napari
import tifffile as tiff
import scimap as sm
import anndata as ad
import pandas as pd

Running SCIMAP  2.3.5


/Users/lukashat/miniforge3/envs/scimap_benchmark/lib/python3.10/site-packages/mpl_scatter_density/__init__.py:4: UserWarning:

pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.

/Users/lukashat/miniforge3/envs/scimap_benchmark/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [3]:
def print_metadata_structure(element, indent=0):
    print("  " * indent + f"Tag: {element.tag}")
    if element.attrib:
        print("  " * (indent + 1) + "Attributes:")
        for key, value in element.attrib.items():
            print("  " * (indent + 2) + f"{key}: {value}")
    for child in element:
        print_metadata_structure(child, indent + 1)

In [19]:
ome_path = '/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/img/S011_chemonaive.ome.tif'
quant = pd.read_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/quantification/cycif_ovca_quantification.csv')
#img = tiff.imread(ome_path)
#seg = tiff.imread('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/IMMUcan/segmentation/IMMUcan_2022_WFLOW_10034294-GI-VAR-TIS-UNST-03_004.tiff')
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/markers.txt') as f:
    channel_names = [line.strip() for line in f.readlines()]

In [20]:
X_columns = quant.columns[:quant.columns.get_loc('x')]
obs_columns = quant.columns[quant.columns.get_loc('x'):]
adata = ad.AnnData(
    X=quant[X_columns],
    obs=quant[obs_columns],
    var=pd.DataFrame(index=X_columns)
)
adata.uns['all_markers'] = channel_names

/Users/lukashat/miniforge3/envs/scimap_benchmark/lib/python3.10/site-packages/anndata/utils.py:304: UserWarning:

X converted to numpy array with dtype float64

/Users/lukashat/miniforge3/envs/scimap_benchmark/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning:

Transforming to str index.



In [21]:
adata

AnnData object with n_obs × n_vars = 2735711 × 33
    obs: 'x', 'y', 'cell_id', 'Patient_code_final', 'sample_id', 'level_1_cell_type', 'level_2_cell_type', 'cell_type'
    uns: 'all_markers'

In [10]:
adata.X

array([[0.37275338, 0.33715615, 0.36710954, ..., 0.3568024 , 0.3646153 ,
        0.3980762 ],
       [0.40103555, 0.5099238 , 0.51247734, ..., 0.5122384 , 0.51237726,
        0.5115856 ],
       [0.3619008 , 0.5038819 , 0.5045507 , ..., 0.5043029 , 0.50587755,
        0.5047233 ],
       ...,
       [0.4852934 , 0.00081594, 0.00162287, ..., 0.00102409, 0.00148179,
        0.00210277],
       [0.4340892 , 0.00065832, 0.00137736, ..., 0.00053539, 0.00144591,
        0.00190976],
       [0.422225  , 0.00052043, 0.0014853 , ..., 0.00061334, 0.00135732,
        0.00214195]])

In [23]:
adata.obs['cell_type'] = adata.obs['cell_type'].astype('category')

In [15]:
adata.obs.dtypes

x                        int64
y                        int64
sample_id                int64
Patient_code_final      object
Sample_code_final       object
cell_type             category
dtype: object

In [22]:
adata.obs

,x,y,cell_id,Patient_code_final,sample_id,level_1_cell_type,level_2_cell_type,cell_type
0,7685,7593,214639,S001,S001_IDS,undefined,undefined,undefined
1,3603,19057,109826,S001,S001_IDS,Cancer,Cancer,Cancer
2,17735,20388,350734,S001,S001_IDS,undefined,undefined,undefined
3,19650,12553,398055,S001,S001_IDS,undefined,undefined,undefined
4,932,13522,6836,S001,S001_IDS,Stromal,Fibroblast,Myofibroblasts
...,...,...,...,...,...,...,...,...
2735706,4097,7476,36244,S010,S010_chemonaive,Cancer,Cancer,Cancer
2735707,5185,7323,44801,S010,S010_chemonaive,Cancer,Cancer,Cancer
2735708,2624,7092,17102,S010,S010_chemonaive,Cancer,Cancer,Cancer
2735709,4855,5271,42468,S010,S010_chemonaive,Immune,Lymphoid_immune,CD8+_T_cell


In [25]:
sm.pl.image_viewer(image_path=ome_path,adata=adata,x_coordinate='y',y_coordinate='x', point_size=5,imageid='sample_id', subset='S011_chemonaive', overlay='cell_type')